# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [58]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć
import numpy as np

class ZnW:
    
    def __init__(self,X):
        self.n = 17
        self.W = [1,0,0,0,1]
        self.max_pow = len(self.W) - 1
        self.X = [num % self.n for num in self.polynomial_remainder(X)]
        self.first_print = True


    def polynomial_remainder(self,dividend):
        divisor = self.W[:]
        
        while len(dividend) >= len(divisor):

            lead_coeff = dividend[0] / divisor[0]
            degree_diff = len(dividend) - len(divisor)
            

            scaled_divisor = [coeff * lead_coeff for coeff in divisor] + [0] * degree_diff
            dividend = [d - s for d, s in zip(dividend, scaled_divisor)]

            while dividend and dividend[0] == 0:
                dividend.pop(0)

            for i in range(len(dividend)):
                if dividend[i] != 0:
                    dividend[i] = int(dividend[i] % self.n)
        
        return dividend
    
    def __add__(self, o):
        len_self = len(self.X)
        len_other = len(o.X)

        if len_self < len_other:
            X_self = [0] * (len_other - len_self) + self.X
            X_other = o.X
        else:
            X_self = self.X
            X_other = [0] * (len_self - len_other) + o.X
        
        res = np.add(X_self,X_other)
        return ZnW(res)

    def __mul__(self,o):
        if isinstance(o,ZnW):
            len_self = len(self.X)
            len_other = len(o.X)

            if len_self < len_other:
                X_self = [0] * (len_other - len_self) + self.X
                X_other = o.X
            else:
                X_self = self.X
                X_other = [0] * (len_self - len_other) + o.X

            return ZnW(np.polymul(X_self,X_other))
        elif isinstance(o,int):
            return ZnW([i * o for i in self.X])
    
    def __rmul__(self, o):
        return self.__mul__(o)
    
    def __len__(self):
        return len(self.X)
    
    def convert_to_nice(self,val,i):
        if val == 0:
            return ""
        if self.first_print:
            symbol = ""
            self.first_print = False
        else:
            symbol = " - " if val < 0 else " + "

        
        if len(self.X) - i - 1 == 0:
            return symbol + str(abs(val))
        
        result = f"x^{len(self.X) - i - 1}"


        if abs(val) == 1:
            return symbol + result
        else:
            return symbol + str(val) + result


    def __str__(self):
        self.first_print = True
        return f"{"".join(self.convert_to_nice(val,i) for i, val in enumerate(self.X))}"
    
    def __repr__(self):
        self.first_print = True
        return f"{"".join(self.convert_to_nice(val,i) for i, val in enumerate(self.X))}"
    

w1 = [7,0,0,14,0,0,0]
w2 = [0,12,10,6]
w3 = [1,1,11,7]

w1 = ZnW(w1)
w2 = ZnW(w2)
w3 = ZnW(w3)

print(f"w1 = {w1}")
print(f"w2 = {w2}")
print(f"w3 = {w3}")

w1 = 14x^3 + 10x^2
w2 = 12x^2 + 10x^1 + 6
w3 = x^3 + x^2 + 11x^1 + 7


## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

In [52]:
K = 2
q = 17

#pierścień Z17[X]/X^4 + 1

prop = 0.1

n = 4

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [76]:
def B_ni_1(prop):
    random_value = np.random.uniform(0,1)

    if random_value < prop:
        return -1
    if random_value < 2*prop:
        return 1
    
    return 0


def test_calculate_t(A,s,e):
    return np.add(np.multiply(A,s), e) 

A = np.array([[ZnW([6,16,16,11]), ZnW([9,4,6,3])],
     [ZnW([5,3,10,11]), ZnW([6,1,9,15])]])

s = np.array([ZnW([-1,-1,1,0]), ZnW([-1,0,-1,0])])

e = np.array([ZnW([0,1,0,0]), ZnW([0,1,-1,0])])

t = test_calculate_t(A,s,e)

print(t[1].X, end="\n\n\n")

def key_gen():
    def generate_A():
        A = [[None for _ in range(K)] for _ in range(K)]
        for i in range(K):
            for j in range(K):
                new_array = np.random.randint(0,q+1,size=n)
                A[i][j] = ZnW(new_array)
        return A
    
    def generate_s():
        s = [None for _ in range(K)]
        for i in range(K):
            new_array = []
            for power in range(n):
                new_array.append(B_ni_1(prop))
            s[i] = ZnW(new_array)
        return s
    
    def generate_e():
        return generate_s()
        

    def calculate_t(A,s,e):
        return np.add(np.multiply(A,s), e) 
    
    A = generate_A()
    s = generate_s()
    e = generate_e()
    t = calculate_t(A,s,e)

    return A,s,t


A,s,t = key_gen()

print(s)
print(t)
print(A)

AttributeError: 'numpy.ndarray' object has no attribute 'X'

### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


### Testy

In [ ]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    A,t,s = key_gen()
    
    m=[sc.choice((0,1)) for k in range(4)]
    
    c = encrypt(A,t,m)
    m_n = decrypt(c,s)

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')
